In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import getpass
if getpass.getuser() == "benjamin.lebrun": sys.path.append("/home/mila/b/benjamin.lebrun/genparse")

In [8]:
import numpy as np
from genparse.util import LarkStuff
from genparse import locally_normalize
from genparse.cfglm import EarleyBoolMaskCFGLM
from arsenal import timeit

In [9]:
cfg = LarkStuff(open("../benchmark/grammars/sql_case_sensitive.lark").read()).char_cfg(.99, ignore='[ ]?')
cfg = locally_normalize(cfg, tol=1e-40, maxiter=np.inf)

with timeit('boolean EarleyCFGLM preprocessing'):
    guide = EarleyBoolMaskCFGLM(cfg) # should take forever


boolean EarleyCFGLM preprocessing (3.2224 sec)


In [ ]:
# try booleanization after unary and nullary removal
# comment out line 48 of earley.py (cfg = cfg.nullaryremove(binarize=True).unarycycleremove().renumber())
# this terminates, but guide.p_next('') is empty.

from genparse.lm import LM
from genparse import EOS, add_EOS, Boolean, Float

class EarleyBoolMaskCFGLM2(LM):
    "LM-like interface for Boolean-masking CFG models; uses Earley's algorithm for inference."

    def __init__(self, cfg):
        from genparse.experimental.earley import Earley
        if EOS not in cfg.V: cfg = add_EOS(cfg)
        cfg = cfg.nullaryremove(binarize=True).unarycycleremove().renumber()
        if cfg.R != Boolean: cfg = cfg.map_values(lambda x: Boolean(x>0), Boolean)
        self.model = Earley(cfg.prefix_grammar)
        super().__init__(eos = EOS, V = cfg.V)

    def p_next(self, context):
        p = self.model.p_next(context).trim()
        return Float.chart({w: 1 for w in p})

    def __call__(self, context):
        assert context[-1] == EOS
        return float(self.model(context) != Boolean.zero)


In [ ]:
guide = EarleyBoolMaskCFGLM2(cfg)

In [ ]:
guide.p_next("") 